# Lab7

---

## Task

Реализовать решение ОДУ сеточным методом.

- Начинать вычисления с грубой сетки (примерно 10 интервалов); измельчать сетку и уточнять по Ричардсону. В идеале — до момента выхода на ошибки округления. Отследить, какая точность (например, от 10<sup>-2</sup> до 10<sup>-6</sup>) достигнута при каком шаге сетки.
- Выводить полученное приближение. Можно на картинке.

# Solution

---

In [ ]:
import math
import numpy as np
from scipy import linalg as la

import matplotlib.pyplot as plt

In [ ]:
def get_error(v1, v2, r = 2, p = 1):
    """
    v1 and v2 are meshes, v1 > v2
    r is the mesh thinchening factor
    p is theoretical order of accuracy
    """
    n, _ = v1.shape
    return np.float_([(v2[2*i] - v1[i]) / (r**p - 1) for i in range(n)])

def solve_grid_eq(p, q, r, f, segment, n, conditions):
    a, b = segment
    h = (b - a) / n

    F = np.zeros((n + 1, 1))
    F[0], F[n] = conditions

    bfunc = np.zeros((n + 1, n + 1))
    bfunc[0, 0], bfunc[n, n] = 1, 1

    grid = np.linspace(a, b, n)
    
    for i in range(1, n):
        x = grid[i]
        bfunc[i, i - 1] = p(x) / h ** 2 - q(x) / (2 * h)
        bfunc[i, i] = -2 * p(x) / h ** 2 - r(x)
        bfunc[i, i + 1] = p(x) / h ** 2  + q(x) / (2 * h)
        F[i] = f(x)

    return la.solve(bfunc, F)

def grid_method(p, q, r, f, segment, conditions, tolerance, max_iter = 20):
    n = 10
    multiplier = 2
    a, b = segment

    v2 = solve_grid_eq(p, q, r, f, segment, n, conditions)
    
    iter = 0
    while max_iter > iter:
        iter += 1
        n = n * multiplier
        v1 = v2
        v2 = solve_grid_eq(p, q, r, f, segment, n, conditions)

        error = get_error(v1, v2)
        if la.norm(error) < tolerance:
            for i in range(len(error)):
                if i % 2 == 0:
                    v2[2*i] += error[i]
                else:
                    v2[i] += (error[i - 1] + error[i + 1]) / 2
            
            m = v2.shape[0]
            x = np.zeros(m, dtype=float)
            h = (b - a)/n
            for i in range(m):
                x[i] = a + i * h / (multiplier ** iter)
            
            return x, v2, h / (multiplier ** iter), iter


## Experimental reseach

In [ ]:
def experiment(x, y):
    plt.grid()
    plt.plot(x, y, marker=".", mec='yellow')
    plt.show()


Вариант 6 из методички

In [ ]:
segment = (-1, 1)
conditions = (0, 0)
eps = [1e-1, 1e-2]

for e in eps:
    x, y, h, iter = grid_method(
        lambda x: (x - 2) / (x + 2), 
        lambda x: x, 
        lambda x: 1 - math.sin(x),
        lambda x: x ** 2,
        segment,
        conditions,
        e
    )

    experiment(x, y, iter)
    print(f"Iterations: {iter}")
    print(f"Mesh step: {h:.10f}")

Вариант 8 из методички

In [ ]:
segment = (-1, 1)
conditions = (0, 0)
eps = [1e-1, 1e-2, 1e-3]

for e in eps:
    x, y, h, iter = grid_method(
        lambda x: -(4 - x) / (5 - 2 * x), 
        lambda x: (1 - x) / 2, 
        lambda x: math.log(3 + x) / 2,
        lambda x: 1 + x / 3,
        segment,
        conditions,
        e
    )
    experiment(x, y, iter)
    print(f"Iterations: {iter}")
    print(f"Mesh step: {h:.10f}")